In [1]:
import pandas as pd
import numpy as np

In [2]:
dislok = pd.read_parquet('dislok_wagons.parquet').convert_dtypes()
dislok = dislok.drop(['plan_date', 'kod_vrab', 'date_pl_rem', 'id_road_disl', 'st_id_dest', 'id_road_dest', 'st_id_send', 'id_road_send', 'last_fr_id'], axis=1)
# dislok = dislok.groupby('wagnum').agg({'date_kap': 'max', 'date_dep': 'max', 'ost_prob': 'min', 'isload': 'mean', 'distance': 'sum'})
print(len(dislok))
dislok.head()

6250933


,wagnum,date_kap,date_dep,ost_prob,isload,fr_id,distance
0,1071,NaT,2020-02-16,36899,0,5032,-1
1,1071,NaT,2020-02-16,36899,1,1204,998
2,1071,NaT,2020-02-16,36711,1,1204,989
3,1071,NaT,2020-02-16,36711,1,1204,939
4,1071,NaT,2020-02-16,36711,1,1204,963


In [3]:
dislok.isna().sum()

wagnum            0
date_kap    2755031
date_dep     351873
ost_prob     257315
isload            0
fr_id             0
distance          0
dtype: int64

In [4]:
freights = pd.read_parquet('freight_info.parquet').convert_dtypes()
freights = pd.get_dummies(freights, columns=['fr_class']).astype(int)
print(len(freights))
freights.head()

5079


,fr_id,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,4989,0,0,0,0,0,0,0,0,0,1,0
1,4990,0,0,0,0,0,0,0,0,0,1,0
2,4991,0,0,0,0,0,0,0,1,0,0,0
3,4992,0,0,0,0,0,0,0,0,0,1,0
4,4993,0,0,0,0,0,0,0,0,0,1,0


In [5]:
merged = pd.merge(dislok, freights, on='fr_id', how='left')
print(len(merged))
merged.head()

6250933


,wagnum,date_kap,date_dep,ost_prob,isload,fr_id,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,NaT,2020-02-16,36899,0,5032,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1071,NaT,2020-02-16,36899,1,1204,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,NaT,2020-02-16,36711,1,1204,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,NaT,2020-02-16,36711,1,1204,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,NaT,2020-02-16,36711,1,1204,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
import sys
sys.path.append('..')
from src.timeconverter import date_to_timestamp

merged['date_kap'] = merged['date_kap'].fillna('1970-01-01')
merged['date_kap'] = merged['date_kap'].astype(str).apply(date_to_timestamp)
merged['date_dep'] = merged['date_dep'].fillna('1970-01-01')
merged['date_dep'] = merged['date_dep'].astype(str).apply(date_to_timestamp)
merged.head()

,wagnum,date_kap,date_dep,ost_prob,isload,fr_id,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,0,1581811200,36899,0,5032,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1071,0,1581811200,36899,1,1204,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,0,1581811200,36711,1,1204,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,0,1581811200,36711,1,1204,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,0,1581811200,36711,1,1204,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [7]:
merged.isna().sum()

wagnum             0
date_kap           0
date_dep           0
ost_prob      257315
isload             0
fr_id              0
distance           0
skoroport     155015
naval         155015
nasip         155015
naliv         155015
openvagons    155015
soprovod      155015
smerz         155015
fr_class_0    155015
fr_class_1    155015
fr_class_2    155015
fr_class_3    155015
dtype: int64

In [8]:
merged.loc[merged['date_kap'] == 0, 'date_kap'] = int(merged.loc[merged['date_kap'] != 0, 'date_kap'].mean())
merged.loc[merged['date_dep'] == 0, 'date_dep'] = int(merged.loc[merged['date_dep'] != 0, 'date_dep'].mean())
merged['ost_prob'] = merged['ost_prob'].fillna(int(merged['ost_prob'].mean()))
merged.head()

,wagnum,date_kap,date_dep,ost_prob,isload,fr_id,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,1498633814,1581811200,36899,0,5032,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1071,1498633814,1581811200,36899,1,1204,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,1498633814,1581811200,36711,1,1204,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,1498633814,1581811200,36711,1,1204,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,1498633814,1581811200,36711,1,1204,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
missings = ['skoroport', 'naval', 'nasip', 'naliv', 'openvagons', 'soprovod', 'smerz', 'fr_class_0', 'fr_class_1', 'fr_class_2', 'fr_class_3']
merged[missings] = merged[missings].fillna(0)
merged.loc[merged['isload'] == 0, missings] = 0
merged = merged.drop('fr_id', axis=1)
merged.head()

,wagnum,date_kap,date_dep,ost_prob,isload,distance,skoroport,naval,nasip,naliv,openvagons,soprovod,smerz,fr_class_0,fr_class_1,fr_class_2,fr_class_3
0,1071,1498633814,1581811200,36899,0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1071,1498633814,1581811200,36899,1,998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1071,1498633814,1581811200,36711,1,989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1071,1498633814,1581811200,36711,1,939,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1071,1498633814,1581811200,36711,1,963,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [ ]:
merged = merged.groupby('wagnum').agg({'date_kap': 'max', 
                                       'date_dep': 'max', 
                                       'ost_prob': 'min', 
                                       'isload': 'mean', 
                                       'distance': 'sum',
                                       'skoroport': 'mean',
                                       'naval': 'mean',
                                       'nasip': 'mean',
                                       'naliv': 'mean',
                                       'openvagons': 'mean',
                                       'soprovod': 'mean',
                                       'smerz': 'mean',
                                       'fr_class_0': 'mean',
                                       'fr_class_1': 'mean',
                                       'fr_class_2': 'mean',
                                       'fr_class_3': 'mean',})